# `precip-dot` data test `01`:
## Consistent estimates in final data

Verify that estimates in the final data (product of warped deltas Atlas 14 data) are consistent.

### inputs

The path to the directory containing the "combined" data to test needs to be specified in the `UNDIFF_DIR` env var prior to running this notebook.

The data will be tested at all valid (non-NaN) grid cells.

In [1]:
def read_data(fp):
    ds = xr.open_dataset(fp)
    ds.close()
    return ds["pf"].values


def test_table(arr):
    """
    check that an array of pf estimates, arranged in the 
    same manner as the NOAA Atlas 14 tables, is consistent 
    """
    intervals_ok = np.all(arr[:, 1:] > arr[:, :-1])
    durations_ok = np.all(arr[1:, :] > arr[:-1, :])
    if intervals_ok and durations_ok:
        return True
    else:
        return False    
   
    
def run_test(data_dir):
    """
    run test on output data directory
    """
    print("Beginning test of consistent estimates in final data.\n")
    # durations to read
    durations = [
        "60m",
        "2h",
        "3h",
        "6h",
        "12h",
        "24h",
        "2d",
        "3d",
        "4d",
        "7d",
        "10d",
        "20d",
        "30d",
        "45d",
        "60d",
    ]

    # template path
    data_fp = os.path.join(data_dir, "pcpt_{}_sum_wrf_{}_{}_undiff.nc")
    
    # test all locations for each future period
    gcms = ["GFDL-CM3", "NCAR-CCSM4"]
    periods = ["2020-2049", "2050-2079", "2080-2099"]
    results = []
    for gcm in gcms:
        for period in periods:
            # construct args for reading data in parallel
            fps = [
                data_fp.format(gcm, duration, period) for duration in durations
            ]
            
            print("Working on data for {}, {}".format(gcm, period))
            print("reading data...")
            # read from each duration in parallel
            p = Pool(15)
            est_arrs = p.map(read_data, fps)
            p.close()
            p.join()

            print("data read, preparing data...")
            # reorder axes to get data aligned properly
            # create an array of arrays, each arranged like the Atlas 14 tables
            read_shp = est_arrs[0].shape
            n = read_shp[1] * read_shp[2]
            r = len(est_arrs)
            c = read_shp[0]
            tables_arr = np.array(est_arrs).swapaxes(1,3).swapaxes(0,2).reshape(n, r, c)
            # subset to only valid (non-NaN) cells
            valid_bool = ~np.isnan(tables_arr[:,0,0])
            valid_tables = tables_arr[valid_bool] # saving tables_arr for possible troubleshooting later

            print("data prepared, testing data...")
            p = Pool(28)
            out = p.map(test_table, valid_tables)
            p.close()
            p.join()

            results.append(np.all(out))
            print("{}, {} complete\n".format(gcm, period))

    final_result = np.all(results)
    # print results
    if final_result:
        print("\nTest result: PASS")
        print("No inconsistencies in estimates found.\n")
    else:
        print("\nTest result: FAIL\n")


In [2]:
import os, time, datetime
import numpy as np
import xarray as xr
from multiprocessing import Pool

data_dir = os.getenv("UNDIFF_DIR")

tic = time.perf_counter()

_ = run_test(data_dir)

print("Elapsed time: {} m\n".format(round((time.perf_counter() - tic) / 60, 1)))

utc_time = datetime.datetime.utcnow()
print("Completion time of previous test: {}".format(utc_time.strftime("%Y-%m-%d %H:%M:%S")))

/home/UA/kmredilla/.localpython/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Beginning test of consistent estimates in final data.

Working on data for GFDL-CM3, 2020-2049
reading data...
data read, preparing data...
data prepared, testing data...
GFDL-CM3, 2020-2049 complete

Working on data for GFDL-CM3, 2050-2079
reading data...
data read, preparing data...
data prepared, testing data...
GFDL-CM3, 2050-2079 complete

Working on data for GFDL-CM3, 2080-2099
reading data...
data read, preparing data...
data prepared, testing data...
GFDL-CM3, 2080-2099 complete

Working on data for NCAR-CCSM4, 2020-2049
reading data...
data read, preparing data...
data prepared, testing data...
NCAR-CCSM4, 2020-2049 complete

Working on data for NCAR-CCSM4, 2050-2079
reading data...
data read, preparing data...
data prepared, testing data...
NCAR-CCSM4, 2050-2079 complete

Working on data for NCAR-CCSM4, 2080-2099
reading data...
data read, preparing data...
data prepared, testing data...
NCAR-CCSM4, 2080-2099 complete


Test result: PASS
No inconsistencies in estimates found.